In [1]:
!pip install delu
!pip install gdown
!pip install rtdl_num_embeddings
!git clone https://github.com/gbulgakov/testing-kan.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing in

In [2]:
# import os
# import shutil

# dir_path = "/kaggle/working/testing-kan"
# if os.path.exists(dir_path):
#     shutil.rmtree(dir_path)
#     print(f"Директория {dir_path} удалена")

In [3]:
# !git clone https://github.com/gbulgakov/testing-kan.git

In [4]:
import sys
import math
import torch
import torch.nn as nn
import numpy as np
from typing import Literal, Optional
import pandas as pd
import numpy as np
import torch
from IPython.display import clear_output
from tqdm import tqdm
import wandb
import pickle
wandb.login(key='936d887040f82c8da3d87f5207c4178259c7b922')

# наши импорты
sys.path.append('/kaggle/working/testing-kan/')
from models.efficient_kan import KAN
from models.fastkan import FastKAN
from models.chebyshev_kan import ChebyKAN
from utils import utils
from utils.tg_bot import send_telegram_file, send_telegram_message
from utils.wandb_tuning import wandb_tuning
from utils.wandb_testing import test_best_model

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gbulgakov (georgy-bulgakov) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [5]:
# !gdown 1tYyhbHdYs_8I9jvXznMoeUAfBzwitaax # house
# !gdown 1hr076cK9QFxH6YZRg5V4av-H7IAve59r # gesture
# !gdown 1GOkNlinj4zHVSNKbqjN1rR4cvsAf2IgR # churn
# !gdown 11B-l4EasJkclK_Q-RBcxYfGJLSvz-v5c # california
!gdown 1p8uqDPMfRlFIc69m7iikS6wGkA6JGj1H # adult

Downloading...
From (original): https://drive.google.com/uc?id=1p8uqDPMfRlFIc69m7iikS6wGkA6JGj1H
From (redirected): https://drive.google.com/uc?id=1p8uqDPMfRlFIc69m7iikS6wGkA6JGj1H&confirm=t&uuid=830bae2f-9b99-4fb1-aff8-3e7960dbecce
To: /kaggle/working/adult.zip
100%|██████████████████████████████████████| 1.12M/1.12M [00:00<00:00, 2.58MB/s]


In [6]:
DATASETS = ['adult', 'california', 'churn', 'covtype', 'eye', 'fb-comments',
           'gesture', 'higgs-small', 'house', 'microsoft', 'otto', 'santander']

REGRESSION = ['house', 'fb-comments', 'microsoft', 'california']
MULTICLASS = ['covtype', 'eye', 'gesture', 'otto']
BINCLASS = ['adult', 'churn', 'higgs-small', 'santander']

In [7]:
def run_single_model(project_name, dataset_name, model_name, arch_type, emb_name, optim_name, dataset, num_epochs, num_trials):
    sweep_id = wandb_tuning(project_name, dataset_name, model_name, arch_type, emb_name, optim_name, dataset, num_epochs, num_trials)
    clear_output(wait=True)
    # вспоминаем лучшие параметры
    api = wandb.Api()
    sweep = api.sweep(f'georgy-bulgakov/{project_name}/{sweep_id}')
    runs = sweep.runs
    best_run = None

    def safe_score(score, direction):
        if isinstance(score, str) or score is None:
            return 1e20 if direction == 'minimize' else 0
        return score

    if dataset['info']['task_type'] == 'regression':
        best_run = min(runs, 
                       key=lambda run : safe_score(run.summary.get('val_loss'), 'minimize'))
    else:
        best_run = max(runs, 
                       key=lambda run : safe_score(run.summary.get('val_acc'), 'maximize'))
    
    best_params = best_run.config
    stats = test_best_model(best_params, project_name, dataset_name, model_name, arch_type, emb_name, optim_name, dataset, num_epochs)
    return stats, best_params

In [8]:
def run_single_dataset(project_name, dataset_name, 
                       optim_names, emb_names, model_names, arch_types,
                       num_epochs, num_trials):
    # dataset_type = dataset_info['type']
    dataset = utils.load_dataset(dataset_name)
    results = []
    pkl_logs = {}

    for model_name in model_names: # можно оставить только kan, тогда model_names = ['kan']
        for arch_type in arch_types:
            for optim_name in optim_names:
                for emb_name in emb_names:
                    stats, best_params = run_single_model(project_name, dataset_name, model_name, arch_type, emb_name, optim_name, dataset, num_epochs, num_trials)
                    results.append(stats)
                    clear_output(wait=True)
                    pkl_logs[f'{model_name}_{arch_type}_{emb_name}_{optim_name}'] = (stats | best_params)
                    send_telegram_message(f'✅ {model_name}_{arch_type}_{emb_name}_{optim_name} finished on {dataset_name}')

    
    with wandb.init(
        project=project_name,
        group=f'dataset_{dataset_name}',
        name='models_comparison'
    ) as run:
        run.log({
            f'final_table_{dataset_name}' : wandb.Table(dataframe=pd.DataFrame(results))
        })

    with open(f'/kaggle/working/{dataset_name}.pkl', 'wb') as f:
        pickle.dump(pkl_logs, f)

    send_telegram_file(f'{dataset_name}.pkl')
    return results



In [ ]:
optim_names = ['adamw']
model_names = ['kan']
emb_names = ['none', 'periodic', 'piecewiselinearq']
arch_types = ['plain', 'tabm', 'tabm-mini']
project_name = 'Debugging KAN TABM'

for dataset_name in ['adult', 'churn', 'gesture', 'california', 'house']:
    run_single_dataset(
        project_name=project_name,
        dataset_name=dataset_name,
        optim_names=optim_names,
        emb_names=emb_names,
        model_names=model_names,
        arch_types=arch_types,
        num_epochs=10,
        num_trials=70
    )

Create sweep with ID: hii0a55m
Sweep URL: https://wandb.ai/georgy-bulgakov/Debugging%20KAN%20TABM/sweeps/hii0a55m


wandb: Agent Starting Run: cj5ieoue with config:
wandb: 	d_embedding: 4
wandb: 	grid_size: 30
wandb: 	kan_layers: 4
wandb: 	kan_width: 1
wandb: 	lr: 0.0050549164386443956
wandb: 	sigma: 3.5351973699870434
wandb: 	weight_decay: 0.0026158795332183863


Traceback (most recent call last):
  File "/kaggle/working/testing-kan/utils/wandb_tuning.py", line 35, in sweep_wrapper
    model = Model(
            ^^^^^^
  File "/kaggle/working/testing-kan/models/tabm_reference.py", line 774, in __init__
    self.num_module = make_module(
                      ^^^^^^^^^^^^
TypeError: models.tabm_reference.make_module() got multiple values for keyword argument 'n_features'


wandb: ERROR Run cj5ieoue errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "/kaggle/working/testing-kan/utils/wandb_tuning.py", line 35, in sweep_wrapper
wandb: ERROR     model = Model(
wandb: ERROR             ^^^^^^
wandb: ERROR   File "/kaggle/working/testing-kan/models/tabm_reference.py", line 774, in __init__
wandb: ERROR     self.num_module = make_module(
wandb: ERROR                       ^^^^^^^^^^^^
wandb: ERROR TypeError: models.tabm_reference.make_module() got multiple values for keyword argument 'n_features'
wandb: ERROR 
wandb: Agent Starting Run: 9btvf0nl with config:
wandb: 	d_embedding: 8
wandb: 	grid_size: 30
wandb: 	kan_layers: 3
wandb: 	kan_width: 4
wandb: 	lr: 0.0013837101212016531
wandb: 	sigma: 33.334089630018894
wandb: 	weight_decay: 3.2748749796217436e-05


Traceback (most recent call last):
  File "/kaggle/working/testing-kan/utils/wandb_tuning.py", line 35, in sweep_wrapper
    model = Model(
            ^^^^^^
  File "/kaggle/working/testing-kan/models/tabm_reference.py", line 774, in __init__
    self.num_module = make_module(
                      ^^^^^^^^^^^^
TypeError: models.tabm_reference.make_module() got multiple values for keyword argument 'n_features'


wandb: ERROR Run 9btvf0nl errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "/kaggle/working/testing-kan/utils/wandb_tuning.py", line 35, in sweep_wrapper
wandb: ERROR     model = Model(
wandb: ERROR             ^^^^^^
wandb: ERROR   File "/kaggle/working/testing-kan/models/tabm_reference.py", line 774, in __init__
wandb: ERROR     self.num_module = make_module(
wandb: ERROR                       ^^^^^^^^^^^^
wandb: ERROR TypeError: models.tabm_reference.make_module() got multiple values for keyword argument 'n_features'
wandb: ERROR 
